# Problem Description.

The task of this project is to write an ASP encoding that solves
the problem of learning an acyclic normal logic program that classifies some data.

The *input* to the learning problem consists of a set of examples, where each example has an identifier,
is labeled as either positive or negative ($\mathit{pos}/\mathit{neg}$), and is associated with a set of Boolean attributes (represented as atoms).

The *input* also includes two additional parameters:
* $\mathit{program\_max\_size}$ fixes the maximum number of rules in the program, and
* $\mathit{body\_max\_size}$ fixes the maximum number of literals in rule bodies.

*Example.*
The following table shows 6 examples over 26 attributes (from 'a' to 'z'):
* example 1 is positive and has attributes c, d, h, i, m, n, and v;
* example 2 is negative and has only attribute m;
* example 3 is positive and has attributes a, c, e, h, j and m; and so on.

Example | Class | a | b | c | d | e | f | g | h | i | j | k | l | m | n | o | p | q | r | s | t | u | v | w | x | y | z 
--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | 
1 | pos | | | X| X| | | | X| X| | | |  X|  X| | | | | | | | X | | | | | 
2 | neg | | |  |  | | | |  |  | | | |  X|   | | | |  | | |    
3 | pos | X| | X| | X| | | X| | X| | | X| | | | | | | |
4 | neg | | | | | | | | | X| | | | | X| | | | | | |    
5 | pos | | | | | | | | | | | X| | | | | | | | | X| X| X|  | |  
6 | neg | | | | X| | | | | | | | | | | | | | |    | |


In an application in medicine,
each example could stand for a person with some attributes,
that is labeled as positive if it has some illness and as negative otherwise.



#### Acyclic normal logic programs

The *atom dependency graph* of a normal logic program $P$ has:
* one node $a$ if either $a$ or $\neg a$ occurs in $P$, and
* one edge from node $b$ to node $a$ if some rule
has the atom $a$ in the head, and either $b$ or $\neg b$ in the body.

We say that a normal logic program is *acyclic* if its atom dependency graph is acyclic.

Note that a normal logic program $P$ is acyclic if and only if it can be written in order, 
i.e., if there is a permutation 
$( r_1, \ldots, r_n)$ of the rules of $P$ such that 
for every $i,j$ such that $1 \leq i \leq j \leq n$, the head of $r_i$ does not occur in the body of $r_j$.

*Examples.* 
The following programs are acyclic (we use semicolons to separate the rules):
* $\{ p \leftarrow \neg q; \ \ \ r \leftarrow p, \neg s \}$
* $\{ p \leftarrow; \ \ \ r \leftarrow s, \neg p; \ \ \ r \leftarrow p, \neg s \}$

while these are not:
* $\{ p \leftarrow \neg q; \ \ \ q \leftarrow \neg p \}$
* $\{ p \leftarrow q; \ \ \ q \leftarrow p  \}$

#### Optimal solutions to a learning problem

Given a learning problem, let:
* $X$ be the set of auxiliary atoms $\{ \mathit{aux}(1), ..., \mathit{aux}(\mathit{program\_max\_size}) \}$, and
* $A$ be the set of attributes occurring in the examples of the problem.

Atom $\mathit{aux}(1)$ represents that an example is classified as positive by the learned program,
while the other auxiliary atoms represent intermediate learned concepts.

*Example.* 
In our running example, assuming $\mathit{program\_max\_size}=4$, we have that:
* $X = \{ \mathit{aux}(1), \mathit{aux}(2), \mathit{aux}(3), \mathit{aux}(4)\}$, and
* $A = \{a,c,d,e,h,i,j,k,m,n,t,u,v\}$.

A normal logic program $P$ is a *possible solution* for such a learning problem if:
* the heads of rules in $P$ belong to $X$,
* the atoms in the bodies of $P$ belong to $X \cup A$ (possibly preceded by negation),
* $P$ is acyclic,
* $P$ has at most $\mathit{program\_max\_size}$ rules, and
* the bodies of $P$ have at most $\mathit{body\_max\_size}$ literals.

*Examples.*
In our running example, assuming $\mathit{body\_max\_size}=2$, we have that:
* rules
  $\mathit{aux(1)} \leftarrow a, \neg c$ and
  $\mathit{aux(3)} \leftarrow i, \neg \mathit{aux}(2)$ may belong to a possible solution,
* while rule
  $\mathit{aux(1)} \leftarrow b, \neg c$, rule
  $\mathit{aux(1)} \leftarrow b, t, \neg c$, and rule 
  $\mathit{aux(5)} \leftarrow i, \neg \mathit{aux}(2)$ do not belong to any possible solution.

A possible solution $P$ classifies an example $E$ *as positive* if
$aux(1)$ belongs to the (unique) stable model of 
$P \cup \{ x \leftarrow \mid E \text{ has attribute }x\}$;
and classifies it *as negative* otherwise.

Note that the program
$P \cup \{ x \leftarrow \mid E \text{ has attribute }x\}$ has exactly one stable model 
because it is acyclic.

*Examples.*
* Program $P_1 = \{ \mathit{aux(1)} \leftarrow h; \ \ \ \mathit{aux(1)} \leftarrow k \}$ 
classifies examples 1, 3 and 5 as positive.
* Program $P_2 = \{ \mathit{aux(1)} \leftarrow h \}$ 
classifies examples 1 and 3 as positive.
* Program $P_3 = \{ \mathit{aux(1)} \leftarrow k; \ \ \ \mathit{aux(1)} \leftarrow m \}$ 
classifies examples 1, 2, 3 and 5 as positive.
* Program $P_4 = \{ \mathit{aux(1)} \leftarrow \neg \mathit{aux(2)}; \ \ \ \mathit{aux(2)} \leftarrow \neg h, \neg u \}$ 
classifies examples 1, 3 and 5 as positive.
* Program $P_5 = \{ \mathit{aux(1)} \leftarrow a; \ \ \ \mathit{aux(1)} \leftarrow c, d; \ \ \ \mathit{aux(1)} \leftarrow k  \}$ 
classifies examples 1, 3 and 5 as positive.


A possible solution $P$ is a *solution* to a given problem 
if:
* it classifies as positive all positive examples of the problem, and
* it classifies as negative all negative examples of the problem.

*Examples.*
Programs $P_1$, $P_4$ and $P_5$ are solutions to our learning problem.

The body size of a logic program $P$ is 
the total sum of the cardinalities of the rule bodies of $P$.

A solution $P$ to a learning problem is *optimal* if:
* there is no other solution with fewer rules, and
* if there are other solutions with the same number of rules,
  then their body size is not smaller than that of $P$.

*Examples.*
* $P_1$ has 2 rules and its body size is 2, $P_4$ has 2 rules and its body size is 3, and
  $P_5$ has 3 rules and its body size is 3.
* $P_5$ is not optimal because there are other solutions, like $P_1$ and $P_4$, with fewer rules.
* $P_4$ is not optimal because there are other solutions, like $P_1$, 
with the same number of rules but smaller body size. 
* $P_1$ is optimal because there are no solutions with one rule, 
and no solution with two rules has body size smaller than 2.


In our application in medicine, the solutions
try to capture the conditions for having the investigated illness.
For instance, optimal solution $P_1$ would suggest that persons having attributes $h$ or $k$ are also ill.

# Representation in ASP.

The examples are represented by facts over the following predicates:    
```
attribute(A).    % A is an attribute
example(E,C).    % example E is of class C (where C can be either pos or neg) 
    has(E,A).    % example E has attribute A
```
Every problem instance also includes constant declarations of the form
```
#const program_max_size=v1.
#const body_max_size=v2.
#const symmetry=v3.
```
where `v1` and `v2` are integers $\geq 1$, and `v3` is either `0` or `1` (see below).

Our running example with `symmetry=0` is represented as follows:
```
#const program_max_size=4.
#const body_max_size=2.
#const symmetry=0.
attribute(a;c;d;e;h;i;j;k;m;n;t;u;v).
example(1,pos). has(1,c). has(1,d). has(1,h). has(1,i). has(1,m). has(1,n). has(1,v).
example(2,neg). has(2,m).
example(3,pos). has(3,a). has(3,c). has(3,e). has(3,h). has(3,j). has(3,m).
example(4,neg). has(4,i). has(4,n).
example(5,pos). has(5,k). has(5,t). has(5,u). has(5,v).
example(6,neg). has(6,d).
```
Observe that the facts over predicate `attribute/1` refer only to attributes that belong to at least one example.

A solution is represented by atoms over the following predicates:  
```
head(R,H).     % the head of rule R is atom H
body(R,S,A).   % the body of rule R has a literal of sign S over atom A
```
Remarks:
* Rules are identified by integers `R`$\geq 1$. 
* Auxiliary atoms $aux(i)$ are represented by their indexes $i$.
  Recall that head atoms `H` can be only auxiliary atoms, while body atoms `A`
  can be either auxiliary atoms or attributes.
* Sign $\texttt{1}$ is used for positive literals, sign $\texttt{-1}$ for negative literals.

*Examples.*

Program $P_1 = \{ \mathit{aux(1)} \leftarrow h; \ \ \ \mathit{aux(1)} \leftarrow k \}$ 
can be represented by:
```
head(1,1). body(1,1,h).
head(2,1). body(2,1,k).
```

Program $P_2 = \{ \mathit{aux(1)} \leftarrow h \}$ 
can be represented by:
```
head(1,1). body(1,1,h).
```

Program $P_3 = \{ \mathit{aux(1)} \leftarrow k; \ \ \ \mathit{aux(1)} \leftarrow m \}$
can be represented by:
```
head(1,1). body(1,1,k).
head(2,1). body(2,1,m).
```

Program $P_4 = \{ \mathit{aux(1)} \leftarrow \neg \mathit{aux(2)}; \ \ \ \mathit{aux(2)} \leftarrow \neg h, \neg u \}$
can be represented by:
```
head(1,1). body(1,-1,2).
head(2,2). body(2,-1,h). body(2,-1,u).
```

Program $P_5 = \{ \mathit{aux(1)} \leftarrow a; \ \ \ \mathit{aux(1)} \leftarrow c, d; \ \ \ \mathit{aux(1)} \leftarrow k  \}$
can be represented by:
```
head(1,1). body(1,1,a).
head(2,1). body(2,1,c). body(2,1,d).
head(3,1). body(3,1,k). 
```

#### Additional constraints

To eliminate equivalent representations of the same solution, we restrict the possible rule representations as follows:
* Every answer must contain fact `head(1,1).`.
* Rule identifiers must be contiguous. For example, in the representation of $P_5$ we cannot replace
  `head(3,1). body(3,1,k).` by `head(4,1). body(4,1,k).` since then we would have rule identifiers
  `1`, `2` and `4`, but no `3`.
* The indexes of the auxiliary atoms must also be contiguous. For example, we cannot represent $P_4$ as follows, since auxiliary atoms would have indexes `1` and `3`, but no `2`:
```
head(1,1). body(1,-1,3).
head(2,3). body(2,-1,h). body(2,-1,u).
``` 
* Auxiliary atoms $\mathit{aux}(i)$ can only appear in the head of rule `R` if all auxiliary atoms
  $\mathit{aux}(j)$ with $1 \leq j < i$:
  * appear in the head of some previous rule, and
  * do not appear in the head of any subsequent rule.
    
  As an example of the first condition, we cannot represent $P_4$ as follows, since $\mathit{aux}(2)$
    would be the head of rule `1` but $\mathit{aux}(1)$ does not appear in any previous rule (there is no rule before rule `1`):
```
  head(1,2). body(1,-1,h). body(1,-1,u).
  head(2,1). body(2,-1,2).
```

<div style="margin-left: 30px;">    
    As an example of the second condition, the following answer is not allowed because rule `2` has head $
    \mathit{aux}(2)$ but the subsequent rule `4` has head $\mathit{aux}(1)$:
</div>

```
   head(1,1). body(1,-1,2).
   head(2,2). body(2,-1,h). body(2,-1,u).
   head(3,2). body(2, 1,h). body(2, 1,u).
   head(4,1). body(3,-1,k).
```


   

#### Additional constraint when symmetry=1

In addition to this, when constant `symmetry` is set to `1`, we add the following constraint,
that enforces an ordering of rules with the same head.

With each rule `R`, we associate a string that contains all atoms `A` such that `body(R,S,A)` holds, ordered according to `clingo`’s lexicographical order. Note that we neglect the sign `S` and only care about the atoms `A`. 

__Examples.__ 
In the representation of $P_4$:
```
head(1,1). body(1,-1,2).
head(2,2). body(2,-1,h). body(2,-1,u).
```
we associate rule `1` with the string `"2"`, and rule `2` with the string `"hu"`.

In the representation of $P_5$:
```
head(1,1). body(1,1,a).
head(2,1). body(2,1,c). body(2,1,d).
head(3,1). body(3,1,k). 
```
we associate rule `1` with `"a"`, rule `2` with `"cd"`, and rule `3` with `"k"`.

If rules `R1` and `R2` such that `R1`$<$`R2` have the same head, the string associated with `R2` must not be lexicographically smaller than the string associated with `R1`. 

__Examples.__ 
The previous representations are fine because for $P_4$ both rules have different heads, 
and for $P_5$ all rules have the same head but `"a"` < `"cd"` < `"k"` according to `clingo`'s lexicographical order.

On the other hand, adding to $P_4$ the rule
```
head(3,2). body(3,1,3). body(3,-1,v).
```
would not be permitted because, for rules `2` and `3`, we would have `"hu"`$\not\lt$`"3v"`. 

Similarly, adding to $P_5$ the rule
```
head(4,1). body(4,1,e).
```
would not be permitted because, for rules `3` and `4`, we would have `"k"`$\not\lt$`"e"`. 

To represent this constraint in ASP, 
you can inspire yourself in the translation of cardinality constraints to normal rules of the course, 
and use some variant of the following rule:
```
% position(A,X): atom A has position X among all atoms in clingo's lexicographical order 
position(A,X) :- atom(A), X = { B <= A : atom(B) }.
```
This predicate can be used to compare strings associated with different rule bodies.


# Visualization

You can use the Python script `asp/viz.py` to visualize the solutions.

For example, you can visualize one solution to our example (`instance-006-4-2-0.lp`) with this command:

In [8]:
! cat asp/solutions/instance-006-4-2-0.json |  python asp/viz.py

aux(1) :- h.
aux(1) :- k.


This is our program $P_1$.

Once you have a working encoding, you can visualize an answer with this command:

In [ ]:
! clingo asp/acyclic-programs.lp asp/instances/instance-006-4-2-0.lp --outf=2 | python asp/viz.py

Option `--outf=2` prints the output in json format. The script `asp/viz.py` prints only the last model in the json file. You may want to modify it to print all models.

# Framework.

The directory ``asp`` contains the files that you need for the project. In the directory ``asp/instances`` you can find the instances. 
Our running example is `instance-006-4-2-0.lp`, where `4`, `2` and `0` are the values of constants
`program_max_size`, `body_max_size`, and `symmetry`, respectively.
In the directory ``asp/solutions`` you can find their solutions in ``json`` format. 

You have to submit a file named ``acyclic-programs.lp``, included as a template in the directory ``asp``, that contains the following lines (and no more ``#show`` statements) so that in the output only the atoms of predicates ``head/2`` and ``body/3`` are shown:

```
#show head/2.
#show body/3.
```

Your encoding should compute all optimal solutions for each instance. 
You can check it by running the ``Python`` script ``test.py`` as follows:
* ``python asp/test.py -e asp/acyclic-programs.lp  -i asp/instances -s asp/solutions -opt -t 100 -m 250``

All instances have less than `250` optimal models, so the script just asks for that number of models.
In fact, all instances except `instance-180-8-4-0.lp` have less than `50` optimal models, so you can start by setting `-m` to that value, and test instance `instance-180-8-4-0.lp` later. 

For help, type `python asp/test.py --help`.

We recommend you to work locally on your computer, using your own installation of ``clingo``.

You can also run your encoding in the next cell. It is not recommended to work in this notebook at ``Binder``, but if you do it, remember to download the files that you modify to your computer, otherwise you will lose your changes.

In [ ]:
%%clingo asp/instances/instance-006-4-2-0.lp -



%
% Display
%

#show head/2.
#show body/3.

# Formalities.

You can work on the solution alone or in groups of two people. 
Different groups have to submit different solutions; in case
of plagiarism, all groups involved will fail the project. 

For every instance, your encoding, combined with the instance, 
must have exactly the optimal models listed in the solution files.
This is tested automatically by the script ``test.py``. 

You can still pass the project if some instances are not solved within
the time limit.

We will send you further instructions about the submission process via Moodle.

# Tips:


* You can try to solve first all instances with option `symmetry=0`. Their names end with `-0.lp`.
  
* Commands to find all optimal stable models look as follows:
```
  clingo asp/acyclic-programs.lp asp/instances/instance-006-4-2-0.lp --opt-mode=optN 0
```
* Option `--quiet=1` avoids printing non-optimal solutions. More clingo options can be found using option `--help=3`.
  
* If you are stuck, you can contact us. We will do our best to answer all your questions. You can send us questions and remarks either via Moodle or by email.

* Start as soon as possible to avoid running out of time. If you later realize that you have problems meeeting the deadline, please contact us instead of copying another solution.